In [2]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
from scipy.stats import loguniform, uniform

# Specify each path

In [3]:
data_directory = "../../../data/"
train_dataset = f'{data_directory}multi_hand_alph_train.csv'
test_dataset = f'{data_directory}multi_hand_alph_test.csv'
model_save_path = './symbol_classifier.keras'
tflite_save_path = './symbol_classifier.tflite'

# Set number of classes

In [4]:
RANDOM_SEED = 42
NUM_CLASSES = 58
MULTI_HAND_LANDMARKS = 126

# Dataset reading

In [5]:
X_train = np.loadtxt(train_dataset, delimiter=',', dtype='float32', usecols=list(range(1, MULTI_HAND_LANDMARKS + 1)))
X_test_data = np.loadtxt(test_dataset, delimiter=',', dtype='float32', usecols=list(range(1, MULTI_HAND_LANDMARKS + 1)))

In [6]:
y_train = np.loadtxt(train_dataset, delimiter=',', dtype='int32', usecols=(0))
y_test_data = np.loadtxt(test_dataset, delimiter=',', dtype='int32', usecols=(0))

In [7]:
X_test, X_valid, y_test, y_valid = train_test_split(X_test_data, y_test_data, train_size=0.75, random_state=RANDOM_SEED)

In [ ]:
print(np.unique(y_train))
print(np.unique(y_test))
print(np.unique(y_valid))

In [ ]:
print(X_train.shape)
print(y_train.shape)

# Model building

In [10]:

def build_fn(input_shape, layers, dropout1, dropout2, dropout3, learning_rate, num_classes=NUM_CLASSES):
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Input((MULTI_HAND_LANDMARKS, )))

    model.add(tf.keras.layers.Dense(layers[0], activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout1))

    model.add(tf.keras.layers.Dense(layers[1], activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout2))

    model.add(tf.keras.layers.Dense(layers[2], activation='relu'))
    model.add(tf.keras.layers.Dropout(dropout3))

    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=learning_rate
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((126, )),
    # tf.keras.layers.BatchNormalization(),  # Normalizes input features
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Dropout for regularization
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),  # Dropout for regularization
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),  # Dropout for regularization
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')  # Output layer with 57 units
])

# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, verbose=1)

In [ ]:
model.summary()

In [ ]:
results = model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_valid, y_valid),
    callbacks=[cp_callback, es_callback]
)

In [ ]:
val_loss, val_accuracy = model.evaluate(X_valid, y_valid)

In [ ]:
def plot_learning_curve(results):
    plt.figure(figsize=(12, 5))
    
    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.ylim(0, 1)
    plt.plot(results.history['loss'], label='Train Loss')
    plt.plot(results.history['val_loss'], label='Validation Loss')
    plt.title('Loss Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.ylim(0, 1)
    plt.plot(results.history['accuracy'], label='Train Accuracy')
    plt.plot(results.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Curve')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

plot_learning_curve(results)

# Hyperparameter Tuning

In [ ]:
model = KerasClassifier(model=build_fn, input_shape=(X_train[1], 1))

param_grid = {
    'model__layers': [
        [128, 64, 32],
        [256, 128, 64]
        # [512, 256, 128],
    ],
    'model__learning_rate': loguniform(1e-4, 1e-2),
    'model__dropout1': uniform(0, 0.5),
    'model__dropout2': uniform(0, 0.5),
    'model__dropout3': uniform(0, 0.5),
    'epochs': [100],
    'batch_size': [32, 64, 128, 256]
}

random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=5, scoring='accuracy', cv=3, random_state=RANDOM_SEED)

search_results = random_search.fit(X_train, y_train, validation_data=(X_valid, y_valid), callbacks=es_callback)

In [ ]:
best_params = search_results.best_params_
best_model = search_results.best_estimator_
best_score = search_results.best_score_

val_accuracy = best_model.score(X_valid, y_valid)

print("Best model Validation accuracy: ", val_accuracy)
print("Best parameters: ", best_params)

In [ ]:
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy: ", test_accuracy)

In [48]:
model = tf.keras.models.load_model(model_save_path)

In [ ]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))
print(y_test[0])

# Confusion matrix

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from scipy.stats import bootstrap as bs
# from bootstrapped import compare_functions as bs_compare

thresholds = np.linspace(0, 1, 100)
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)
f1_scores = [f1_score(y_test, y_pred > threshold, average='weighted') for threshold in thresholds]

bootstrap_results = bs(data=np.array(f1_scores), np.std, method='percentile', confidence_level=0.05)

lower_bound = bootstrap_results.lower_bound
upper_bound = bootstrap_results.upper_bound

plt.plot(thresholds, f1_scores, label='F1 Score')
plt.fill_between(thresholds, lower_bound, upper_bound, alpha=0.3, label='95% CI')
plt.xlabel('Threshold')
plt.ylabel('F1 Score')
plt.title('F1 Confidence Curve')
plt.legend()
plt.show()

# Convert to model for Tensorflow-Lite

In [51]:
# Save as a model dedicated to inference
model.save(model_save_path)

In [ ]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

In [20]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [21]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))